## Urban Heat Island (UHI) effect reduction 
### Written by: Amy Tran

## Scenario

## Objective

# Datasets
trees-with-species-and-dimensions-urban-forest<br>
laneways-with-greening-potential<br>
microclimate-sensor-readings<br>

## Codes

#### Load data 

In [299]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import plotly.express as px


#pip install plotly==5.8.0

In [198]:
#Set api key
#API_KEY = ''

API_KEY = '170b7e637bb535ddd36f2ad975e8efab6de6d42f477b543b95e2e838'
#headers = {}
#headers['Accept'] = 'application/json; charset=utf-8json'
#headers['Authorization'] = f"Bearer {API_KEY}"
#headers

session = requests.Session()
#session.headers.update(headers)

In [206]:
#Sample query
#import requests
#session = requests.Session()
#base = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'
#url = 'microclimate-sensor-readings'
#filters = f'records?limit={10}&offset={9990}&timezone=UTC'
#filters = f'records?limit={10}&offset={0}&timezone=UTC'
#target_url = f'{base}{url}/{filters}'
#result = session.get(target_url+f'&apikey={API_KEY}')
#result

<Response [400]>

In [200]:
#Create function to make iterative calls 
def get_data(base, url, size = 0):
    #Extract 1st row as default result 
    #default_filters = filters = f'records?limit={1}&offset={0}&timezone=UTC'
    #default_url = f'{base}{url}/{default_filters}'
    #default_result = session.get(default_url)
    #default_result_json = default_result_json()
    #default_max_results = default_result_json['total_count']

    #Extract target result (full data)
    target_filters = f'records?limit={10}&offset={size}&timezone=UTC'
    target_url = f'{base}{url}/{target_filters}'
    result = session.get(target_url+f'&apikey={API_KEY}')
    status_code = result.status_code
    if status_code == 200:
        result_json = result.json()
        max_results = result_json['total_count']
        links = result_json['links']
        records = result_json['records']
        records_df = pd.json_normalize(records)
    
        #Update column labels
        records_df.drop(columns=['links'],inplace=True)
        column_names = records_df.columns.values.tolist()

        #Replace geolocation.lat & geolocation.lon
        column_names = ['_'.join((a.split(".")[-2:])) if a.split('.')[-2]=='geolocation' else a for a in column_names]
        column_names = [i.split('.')[-1] for i in column_names]
        records_df.columns = column_names
    
        next_url = None
             
        #Obtain next url
        if records_df.shape[0] != max_results:
            for l in links:
                if l['rel'] == 'next':
                    next_url = l['href']
    
        return[records_df, next_url, column_names, max_results, status_code]
    else: return[None, None, None, None, status_code]

In [202]:
#Extract urban forest data 
url_next = ''
df = pd.DataFrame()
base = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'
url = 'microclimate-sensor-readings'
status_code = 200
max_results = 0

#while (df.shape[0] < 60) and (status_code == 200):
while (url_next != None) and (status_code == 200):
    df_new, url_next, column_names, max_results, status_code = get_data(base, url, df.shape[0])
    if status_code == 200:
        df = df.append(df_new, ignore_index=True)
        print('Status code: ' + str(status_code) + ' Total no of records: ' + str(max_results)  + ' No of rows extracted: ' + str(df.shape[0]))
    else: 
        print('Status code: ' + str(status_code) + ' Extraction ceased')
        break

msr_df = df.copy()
msr_df.head()
#msr_df.shape    
    

Status code: 200 Total no of records: 459859 No of rows extracted: 10
Status code: 200 Total no of records: 459859 No of rows extracted: 20
Status code: 200 Total no of records: 459859 No of rows extracted: 30
Status code: 200 Total no of records: 459859 No of rows extracted: 40
Status code: 200 Total no of records: 459859 No of rows extracted: 50
Status code: 200 Total no of records: 459859 No of rows extracted: 60
Status code: 200 Total no of records: 459859 No of rows extracted: 70
Status code: 200 Total no of records: 459859 No of rows extracted: 80
Status code: 200 Total no of records: 459859 No of rows extracted: 90
Status code: 200 Total no of records: 459859 No of rows extracted: 100
Status code: 200 Total no of records: 459859 No of rows extracted: 110
Status code: 200 Total no of records: 459859 No of rows extracted: 120
Status code: 200 Total no of records: 459859 No of rows extracted: 130
Status code: 200 Total no of records: 459859 No of rows extracted: 140
Status code: 20

In [293]:
#Filter temperature records only
temp_data = msr_df.loc[msr_df['type'] == 'TPH.TEMP']
#Convert local_time from string to datetime 
temp_data['read_date'] = temp_data['local_time'].astype(str).str[:10]
temp_data['read_date'] = pd.to_datetime(temp_data['read_date'], format = '%Y-%m-%d')
#temp_data.head()
#temp_data.tail()
#temp_data.dtypes

<ipython-input-293-41fe39f356e3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['read_date'] = temp_data['local_time'].astype(str).str[:10]
<ipython-input-293-41fe39f356e3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data['read_date'] = pd.to_datetime(temp_data['read_date'], format = '%Y-%m-%d')


In [294]:
temp_data.groupby(['site_id'])['value'].mean()

site_id
1011    17.700286
1012    17.667962
1013    20.571545
1014    18.440476
1016    16.369121
Name: value, dtype: float64

In [301]:
#Extract Block level energy consumption projection for 2026
url_next = ''
df = pd.DataFrame()
base = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'
url = 'block-level-energy-consumption-modelled-on-building-attributes-2026-projection-r'
status_code = 200
max_results = 0

#while (df.shape[0] < 60) and (status_code == 200):
while (url_next != None) and (status_code == 200):
    df_new, url_next, column_names, max_results, status_code = get_data(base, url, df.shape[0])
    if status_code == 200:
        df = df.append(df_new, ignore_index=True)
        print('Status code: ' + str(status_code) + ' Total no of records: ' + str(max_results)  + ' No of rows extracted: ' + str(df.shape[0]))
    else: 
        print('Status code: ' + str(status_code) + ' Extraction ceased')
        break

energy_df = df.copy()
energy_df.head()
#energy_df.shape
    

Status code: 200 Total no of records: 628 No of rows extracted: 10
Status code: 200 Total no of records: 628 No of rows extracted: 20
Status code: 200 Total no of records: 628 No of rows extracted: 30
Status code: 200 Total no of records: 628 No of rows extracted: 40
Status code: 200 Total no of records: 628 No of rows extracted: 50
Status code: 200 Total no of records: 628 No of rows extracted: 60
Status code: 200 Total no of records: 628 No of rows extracted: 70
Status code: 200 Total no of records: 628 No of rows extracted: 80
Status code: 200 Total no of records: 628 No of rows extracted: 90
Status code: 200 Total no of records: 628 No of rows extracted: 100
Status code: 200 Total no of records: 628 No of rows extracted: 110
Status code: 200 Total no of records: 628 No of rows extracted: 120
Status code: 200 Total no of records: 628 No of rows extracted: 130
Status code: 200 Total no of records: 628 No of rows extracted: 140
Status code: 200 Total no of records: 628 No of rows extr

,id,timestamp,size,lon,lat,type,coordinates,type,total
0,50e083ed2416e3c206e144509a10ce76a0416135,2022-11-13T10:03:59.153Z,323,144.984907,-37.843406,Feature,"[[[[144.98541694518966, -37.84319604667665], [...",MultiPolygon,56.160850
1,d339e27fc4cfb703b499a5d7a0d7033969e2f494,2022-11-13T10:03:59.153Z,369,144.977897,-37.836157,Feature,"[[[[144.97844876361296, -37.835273948380305], ...",MultiPolygon,289.290570
2,41129e571a58276ecea18806c67e126b73b77c8e,2022-11-13T10:03:59.153Z,1649,144.975948,-37.835444,Feature,"[[[[144.97632715219194, -37.83606901176402], [...",MultiPolygon,3229.163202
3,cb01754a02ac0afe247f303ab4b40b6c307a33f6,2022-11-13T10:03:59.153Z,448,144.984540,-37.836237,Feature,"[[[[144.98536190559653, -37.83473604011838], [...",MultiPolygon,730.062456
4,2532f4438f1c29c209d363b699843d770778be00,2022-11-13T10:03:59.153Z,2371,144.981260,-37.835787,Feature,"[[[[144.9822490509734, -37.834943284170805], [...",MultiPolygon,2783.788766


In [300]:
#Heatmap for energy consumption projection
fig = px.density_mapbox(energy_df, lat = 'lat', lon = 'lon', z = 'total', mapbox_style="stamen-terrain")
fig